## 3. Predicción de fugas de clientes en una compañía telefónica

Una compañía telefónica está interesada en que desarrollemos un modelo que prediga los **100 clientes actuales** (dataset de explotaación) que tienen más probabilidad de abandonar la compañía. Para ello nos proporcionan una base de datos **fuga_clientes_empresa_telefonica_construccion.csv** con casos etiquetados, que usaremos para construir nuestro modelo de predicción.

Los campos de esta base de datos son:

* **Customer ID**

* **network_age**: antigüedad del cliente en días

* **Customer tenure in months:** antigüedad del cliente en meses

* **Total Spend in Months 1 and 2:** gasto total del cliente en los meses de referencia 1 y 2

* **Total SMS Spend:** gasto total en SMS

* **Total Data Spend:** gasto total en datos/internet

* **Total Data Consumption:** consumo total de datos (en KB) durante el período de estudio

* **Total Unique Calls:** número total de llamadas únicas

* **Total Onnet spend:** gasto total en llamadas a otros usuarios de la misma red de telefonía

* **Total Offnet spend:** gasto total en llamadas a otros usuarios de redes de telefonía diferentes

* **Total Call centre complaint calls:** número de llamadas de queja al call center

* **Network type subscription in Month 1:** suscripción de tipo de red en el mes 1. Esto indica la suscripción de red preferida de un cliente, lo que puede indicar su tipo de dispositivo: servicio 2G o 3G

* **Network type subscription in Month 2:** igual que el anterior pero en el mes posterior

* **Churn Status**: el valor es 1 si el cliente abandona la compañía telefónica, 0 si permanece en ella

* **Most Loved Competitor network in Month 1:** qué otro proveedor de la competencia prefiere el cliente en el mes 1. En realidad es un conjunto de columnas, cada una enfocada en un proveedor particular

* **Most Loved Competitor network in Month 2:** qué otro proveedor de la competencia prefiere el cliente en el mes 2. En realidad es un conjunto de columnas, cada una enfocada en un proveedor particular

La variable a predecir es **Churn Status**: el valor es 1 si el cliente **abandona** la compañía, 0 si no la abandona.

La compañía también nos proporciona otra base de datos, **fuga_clientes_empresa_telefonica_explotacion.csv**, con información sobre clientes de los que no se sabe ahora mismo si van a permanecer o no en la compañía. Por tanto en esta segunda base de datos todos los valores de la columna **Churn Status** son missing values (NaN).

La compañía nos pide que proporcionemos los IDs de los 100 clientes de la base de datos de explotación que con mayor probabilidad vayan a abandonar la compañía. Para ello proporcionaremos como entregable a la compañía un archivo csv con una sola columna, **Customer ID**, y 100 filas que contengan los IDs de los clientes seleccionados.

El fichero **ejemplo_fichero_predicciones.csv** contiene un ejemplo con el formato solicitado para este archivo.

In [53]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB

In [2]:
datos_construccion = pd.read_csv("./fuga_clientes_empresa_telefonica_construccion.csv")
datos_explotacion  = pd.read_csv("./fuga_clientes_empresa_telefonica_explotacion.csv")

In [3]:
datos_construccion.columns

Index(['Customer ID', 'network_age', 'Customer tenure in month',
       'Total Spend in Months 1 and 2', 'Total SMS Spend', 'Total Data Spend',
       'Total Data Consumption', 'Total Unique Calls', 'Total Onnet spend',
       'Total Offnet spend', 'Total Call centre complaint calls',
       'Churn Status', 'Most Loved Competitor network in Month 1_0',
       'Most Loved Competitor network in Month 1_Mango',
       'Most Loved Competitor network in Month 1_PQza',
       'Most Loved Competitor network in Month 1_ToCall',
       'Most Loved Competitor network in Month 1_Uxaa',
       'Most Loved Competitor network in Month 1_Weematel',
       'Most Loved Competitor network in Month 1_Zintel',
       'Most Loved Competitor network in Month 2_Mango',
       'Most Loved Competitor network in Month 2_PQza',
       'Most Loved Competitor network in Month 2_ToCall',
       'Most Loved Competitor network in Month 2_Uxaa',
       'Most Loved Competitor network in Month 2_Weematel',
       'Most 

In [4]:
datos_explotacion.columns

Index(['Customer ID', 'network_age', 'Customer tenure in month',
       'Total Spend in Months 1 and 2', 'Total SMS Spend', 'Total Data Spend',
       'Total Data Consumption', 'Total Unique Calls', 'Total Onnet spend',
       'Total Offnet spend', 'Total Call centre complaint calls',
       'Churn Status', 'Most Loved Competitor network in Month 1_0',
       'Most Loved Competitor network in Month 1_Mango',
       'Most Loved Competitor network in Month 1_PQza',
       'Most Loved Competitor network in Month 1_ToCall',
       'Most Loved Competitor network in Month 1_Uxaa',
       'Most Loved Competitor network in Month 1_Weematel',
       'Most Loved Competitor network in Month 1_Zintel',
       'Most Loved Competitor network in Month 2_Mango',
       'Most Loved Competitor network in Month 2_PQza',
       'Most Loved Competitor network in Month 2_ToCall',
       'Most Loved Competitor network in Month 2_Uxaa',
       'Most Loved Competitor network in Month 2_Weematel',
       'Most 

In [5]:
# chequeo de que las columnas son exactamente las mismas en los dos ficheros:

sum(datos_construccion.columns != datos_explotacion.columns)

0

In [6]:
# la columna a predecir es "Churn Status"
# si es 1, el cliente se va de la compañía
# si es 0, el cliente se queda

# Esta columna se sabe en el dataset de construcción (ejemplos de clientes pasados):
datos_construccion["Churn Status"].values[:10]

array([0, 0, 0, 0, 0, 0, 0, 1, 1, 1])

In [7]:
# Sin embargo no se sabe en el dataset de explotación (clientes actuales):
datos_explotacion["Churn Status"].values[:10]

array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan])

In [26]:
dc_attrs = list(datos_construccion.columns)
dc_attrs.remove('Churn Status')
dc_attrs.remove('Customer ID')
X = datos_construccion[dc_attrs].values
Y = datos_construccion['Churn Status'].values

In [220]:
clf = DecisionTreeClassifier(max_depth=5)
clf = clf.fit(X, Y)
scores = cross_val_score(clf, X, Y,cv=5)
print("Score global del modelo: {:.2f} +/- {:.2f}".format(scores.mean(), scores.std()))

Score global del modelo: 0.77 +/- 0.07


In [222]:
knc = KNeighborsClassifier(n_neighbors=30)
knc = knc.fit(X, Y)
scores = cross_val_score(knc, X, Y, cv=5)
print("Score global del modelo: {:.2f} +/- {:.2f}".format(scores.mean(), scores.std()))

Score global del modelo: 0.64 +/- 0.06


In [225]:
mlpc = MLPClassifier(hidden_layer_sizes=(10,10,), max_iter=100000,
                              alpha=0.1)
mlpc = mlpc.fit(X, Y)
scores = cross_val_score(mlpc, X, Y, cv=5)
print("Score global del modelo: {:.2f} +/- {:.2f}".format(scores.mean(), scores.std()))

Score global del modelo: 0.57 +/- 0.03


In [160]:
gnb = GaussianNB()
gnb = gnb.fit(X, Y)
scores = cross_val_score(gnb, X, Y, cv=5)
print("Score global del modelo: {:.2f} +/- {:.2f}".format(scores.mean(), scores.std()))

Score global del modelo: 0.62 +/- 0.02


Tras estudiar las puntuaciones de los distintos clasificadores con distintos parámetros, hemos obtenido que el mejor de todos es el árbol de decisión con profuncidad máxima 5. A continuación, vamos a calcular las probabilidades de que el atributo 'Churn Status' de los datos de explotación sea 1 a partir de la función *predict_proba*, que devuelve la probabilidad de cada valor de la clase (hemos asumido que el orden de las tuplas es \[prob_0, prob_1\]). Después, se ordenan de manera que queden ordenadas de mayor a menor según la probabilidad de que se vayan ('Churn Status' = 1), correspondemos con los datos de *datos_explotacion* e imprimimos los 100 primeros IDs.

In [221]:
probs = clf.predict_proba(datos_explotacion[dc_attrs])
datos_completos[np.argsort(probs[:, 0])][:, 0]

array(['ADF1811', 'ADF1972', 'ADF1806', 'ADF1602', 'ADF0524', 'ADF0127',
       'ADF0137', 'ADF0561', 'ADF1060', 'ADF1841', 'ADF1821', 'ADF1817',
       'ADF1955', 'ADF1772', 'ADF1815', 'ADF1774', 'ADF1829', 'ADF1823',
       'ADF1780', 'ADF1803', 'ADF1775', 'ADF1641', 'ADF1773', 'ADF0444',
       'ADF0451', 'ADF1978', 'ADF0513', 'ADF0511', 'ADF0098', 'ADF0480',
       'ADF0081', 'ADF1896', 'ADF1638', 'ADF0409', 'ADF1901', 'ADF0442',
       'ADF1623', 'ADF0484', 'ADF1632', 'ADF1598', 'ADF0391', 'ADF1596',
       'ADF1579', 'ADF1572', 'ADF0354', 'ADF1975', 'ADF0621', 'ADF0607',
       'ADF0603', 'ADF0601', 'ADF1597', 'ADF0363', 'ADF1560', 'ADF0458',
       'ADF0517', 'ADF1563', 'ADF1564', 'ADF1567', 'ADF1582', 'ADF0580',
       'ADF1561', 'ADF0420', 'ADF0447', 'ADF1985', 'ADF1650', 'ADF1631',
       'ADF1613', 'ADF1612', 'ADF1593', 'ADF1581', 'ADF1555', 'ADF1063',
       'ADF1657', 'ADF1590', 'ADF1601', 'ADF1607', 'ADF0494', 'ADF1615',
       'ADF1616', 'ADF1617', 'ADF1620', 'ADF1606', 